In [1]:
import sys
import pandas as pd
import fastparquet
from torch.utils.data import DataLoader, Dataset
import torch
import random
import pickle
import numpy as np
from collections import defaultdict
from itertools import chain
import re
from torch.nn.utils.rnn import pad_sequence
import json

sys.path.append('../')

with open('../backend/config.json', 'r') as f:
    config = json.load(f)

# --- Data Loading and Preprocessing ---
def load_and_process_parquet(path):
    """Load a parquet file and create triplets with correct positive/negative logic."""
    print(f"\nProcessing {path}...")
    df = pd.read_parquet(path, engine='fastparquet')

    # Filter for valid rows: keep queries that have at least one passage
    valid_mask = (
        df['query'].notna() & 
        df['passages.passage_text'].notna() &
        df['passages.passage_text'].apply(lambda x: len(x) > 0 if isinstance(x, list) else False)
    )
    df = df[valid_mask].reset_index(drop=True)
    print(f"  Found {len(df)} valid queries.")

    # Create a flat list of (query_id, passage) for negative sampling
    all_passages = []
    for idx, row in df.iterrows():
        for p in row['passages.passage_text']:
            all_passages.append((idx, p))  # tag with query index for filtering later

    triplets = []
    for idx, row in df.iterrows():
        query = row['query']
        query_passages = row['passages.passage_text']

        for _ in range(config['NUM_TRIPLETS_PER_QUERY']): 
            positive = random.choice(query_passages)

            # Negative must be from a *different* query
            while True:
                neg_query_id, negative = random.choice(all_passages)
                if neg_query_id != idx:
                    break

            triplets.append({'query': query, 'positive': positive, 'negative': negative})

    print(f"  Generated {len(triplets)} triplets.")
    # Convert to list of tuples for TripletDataset
    return [(t['query'], t['positive'], t['negative']) for t in triplets]

# --- Tokenizer and Vocab ---
class PretrainedTokenizer:
    def __init__(self, word_to_idx_path):
        # Load pretrained word_to_idx mapping
        with open(word_to_idx_path, 'rb') as f:
            self.word2idx = pickle.load(f)
        
        print(f"Loaded vocabulary with {len(self.word2idx):,} tokens")

    def encode(self, sentence):
        # Tokenize sentence, preserving punctuation.
        tokens = re.findall(r"\w+|[.,!?;]", str(sentence))
        # Only include words that exist in vocabulary, skip unknown words
        return [self.word2idx[word] for word in tokens if word in self.word2idx]

    def vocab_size(self):
        return len(self.word2idx)

# --- Dataset Class ---
class TripletDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        query, pos_doc, neg_doc = self.data[idx]
        return (torch.tensor(self.tokenizer.encode(query), dtype=torch.long),
                torch.tensor(self.tokenizer.encode(pos_doc), dtype=torch.long),
                torch.tensor(self.tokenizer.encode(neg_doc), dtype=torch.long))

# --- Collate Function ---
def collate_fn(batch):
    queries, pos_docs, neg_docs = zip(*batch)
    return (
        pad_sequence(queries, batch_first=True, padding_value=0),
        pad_sequence(pos_docs, batch_first=True, padding_value=0),
        pad_sequence(neg_docs, batch_first=True, padding_value=0)
    )

In [2]:
# Load pretrained tokenizer
tokenizer = PretrainedTokenizer('../data/word_to_idx.pkl')

# Dataset paths
dataset_paths = {
    'train': '../data/ms_marco_train.parquet',
    'validation': '../data/ms_marco_validation.parquet',
    'test': '../data/ms_marco_test.parquet'
}

# Create datasets and dataloaders
dataloaders = {}
datasets = {}

print("\nCreating DataLoaders...")
for split, path in dataset_paths.items():
    data_tuples = load_and_process_parquet(path)
    datasets[split] = TripletDataset(data_tuples, tokenizer)
    
    # Use a smaller batch size for validation and test if needed
    batch_size = 2 # You can adjust this

    dataloaders[split] = DataLoader(
        datasets[split],
        batch_size=batch_size,
        shuffle=(split == 'train'),
        num_workers=0,  # Set to 0 for Jupyter notebooks to avoid multiprocessing issues
        collate_fn=collate_fn,
        pin_memory=False  # Set to False for MPS compatibility
    )
    print(f"✅ {split} dataloader ready!")

# Extract data for later use (e.g., evaluation)
train_data = datasets['train'].data
val_data = datasets['validation'].data
test_data = datasets['test'].data

print(f"\nDataset sizes:")
print(f"  Training: {len(train_data):,} triplets")
print(f"  Validation: {len(val_data):,} triplets")
print(f"  Test: {len(test_data):,} triplets")

Loaded vocabulary with 400,000 tokens

Creating DataLoaders...

Processing ../data/ms_marco_train.parquet...
  Found 808731 valid queries.
  Generated 808731 triplets.
✅ train dataloader ready!

Processing ../data/ms_marco_validation.parquet...
  Found 101093 valid queries.
  Generated 101093 triplets.
✅ validation dataloader ready!

Processing ../data/ms_marco_test.parquet...
  Found 101092 valid queries.
  Generated 101092 triplets.
✅ test dataloader ready!

Dataset sizes:
  Training: 808,731 triplets
  Validation: 101,093 triplets
  Test: 101,092 triplets


In [3]:
# Check if MPS is available
print(torch.backends.mps.is_available())

True


In [4]:
# Test the training dataloader
train_loader = dataloaders['train']

print(f"📊 Dataset Info:")
print(f"  Total batches in train: {len(train_loader)}")
print(f"  Batch size: {train_loader.batch_size}")

print("\n🔍 Testing first few batches...")
for batch_idx, (query_batch, pos_batch, neg_batch) in enumerate(train_loader):
    print(f"\nBatch {batch_idx + 1}:")
    print(f"  Query batch shape: {query_batch.shape}")
    print(f"  Positive batch shape: {pos_batch.shape}")
    print(f"  Negative batch shape: {neg_batch.shape}")
    
    if batch_idx >= 3:  # Just show first 4 batches
        break

print("\n✅ DataLoader test completed!")

📊 Dataset Info:
  Total batches in train: 404366
  Batch size: 2

🔍 Testing first few batches...

Batch 1:
  Query batch shape: torch.Size([2, 5])
  Positive batch shape: torch.Size([2, 55])
  Negative batch shape: torch.Size([2, 53])

Batch 2:
  Query batch shape: torch.Size([2, 6])
  Positive batch shape: torch.Size([2, 49])
  Negative batch shape: torch.Size([2, 80])

Batch 3:
  Query batch shape: torch.Size([2, 12])
  Positive batch shape: torch.Size([2, 29])
  Negative batch shape: torch.Size([2, 50])

Batch 4:
  Query batch shape: torch.Size([2, 8])
  Positive batch shape: torch.Size([2, 81])
  Negative batch shape: torch.Size([2, 49])

✅ DataLoader test completed!


In [5]:
import torch
import torch.nn as nn

# --- Dual RNN Encoder Model ---
class RNNEncoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, pretrained_embeddings=None):
        super().__init__()
        # Use padding_idx=0 since we pad with 0
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        
        # Load pretrained embeddings if provided
        if pretrained_embeddings is not None:
            self.embedding.weight.data.copy_(torch.from_numpy(pretrained_embeddings))
            # Keep embeddings trainable (they are by default)
            
        self.rnn = nn.GRU(embed_dim, hidden_dim, batch_first=True)

    def forward(self, x):
        x = self.embedding(x)
        _, h_n = self.rnn(x)
        return h_n.squeeze(0)  # shape: (batch, hidden_dim)

# --- Triplet Loss Function ---
def triplet_loss_function(triplet, distance_function, margin):
    query, pos_doc, neg_doc = triplet
    d_pos = distance_function(query, pos_doc)
    d_neg = distance_function(query, neg_doc)
    return torch.clamp(d_pos - d_neg + margin, min=0.0).mean()

In [6]:
# Load pretrained tokenizer
tokenizer = PretrainedTokenizer('../data/word_to_idx.pkl')

Loaded vocabulary with 400,000 tokens


In [7]:
import os
import gc
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import random
import json
import numpy as np

# 🔧 SET MPS MEMORY ENVIRONMENT VARIABLE
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'
print("🔧 Set PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 (disables memory limit)")

# 🧹 MEMORY MANAGEMENT UTILITIES
def clean_memory():
    """Aggressive memory cleaning for MPS/CUDA"""
    gc.collect()  # Python garbage collection
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    if torch.backends.mps.is_available():
        try:
            torch.mps.empty_cache()
            torch.mps.synchronize()
        except AttributeError:
            # Older PyTorch versions might not have these methods
            pass
    print("🧹 Memory cleaned")

def get_memory_usage():
    """Get current memory usage"""
    if torch.cuda.is_available():
        return f"CUDA: {torch.cuda.memory_allocated()/1024**3:.2f}GB / {torch.cuda.memory_reserved()/1024**3:.2f}GB"
    elif torch.backends.mps.is_available():
        # MPS doesn't have direct memory query methods, so we return a placeholder
        return "MPS: Memory tracking not available"
    return "CPU: Memory tracking not available"

# Clean memory before starting
clean_memory()

with open('../backend/config.json', 'r') as f:
    config = json.load(f)

# --- Training Setup ---
VOCAB_SIZE = tokenizer.vocab_size()

# Load pretrained embeddings
pretrained_embeddings = np.load('../data/embeddings.npy')
EMBED_DIM = pretrained_embeddings.shape[1]  # Get embedding dimension from loaded embeddings

print(f"Loaded pretrained embeddings: {pretrained_embeddings.shape}")
print(f"Vocabulary size: {VOCAB_SIZE}")
print(f"Embedding dimension: {EMBED_DIM}")

# Enhanced device selection with better fallback handling
def get_best_device():
    """Get the best available device with informative feedback."""
    if torch.backends.mps.is_available():
        print("✅ MPS (Apple Silicon GPU) is available and will be used")
        return torch.device('mps')
    elif torch.cuda.is_available():
        print("✅ CUDA GPU is available and will be used")
        return torch.device('cuda')
    else:
        print("⚠️  No GPU acceleration available - using CPU")
        print("   💡 For better performance on Apple Silicon, run outside Docker")
        print("   💡 For CUDA support, configure Docker with GPU passthrough")
        return torch.device('cpu')

device = get_best_device()
print(f"Selected device: {device}")
print(f"Memory before model loading: {get_memory_usage()}")

# Initialize encoders with pretrained embeddings and move to GPU
print("🔄 Loading models to device...")
clean_memory()  # Clean before loading models

query_encoder = RNNEncoder(VOCAB_SIZE, EMBED_DIM, config['HIDDEN_DIM'], pretrained_embeddings).to(device)
print(f"✅ Query encoder loaded. Memory: {get_memory_usage()}")

doc_encoder = RNNEncoder(VOCAB_SIZE, EMBED_DIM, config['HIDDEN_DIM'], pretrained_embeddings).to(device)
print(f"✅ Doc encoder loaded. Memory: {get_memory_usage()}")

optimizer = torch.optim.Adam(list(query_encoder.parameters()) + list(doc_encoder.parameters()), lr=config['LR'])

# The dataloader is already created, we just need to adjust batch size in the config
# and re-run the dataloader creation cell if we want to change it.
config['BATCH_SIZE'] = dataloaders['train'].batch_size
train_loader = dataloaders['train']

clean_memory()  # Final cleanup
print(f"🚀 Ready to train! Final memory: {get_memory_usage()}")

🔧 Set PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 (disables memory limit)
🧹 Memory cleaned
Loaded pretrained embeddings: (400000, 200)
Vocabulary size: 400000
Embedding dimension: 200
✅ MPS (Apple Silicon GPU) is available and will be used
Selected device: mps
Memory before model loading: MPS: Memory tracking not available
🔄 Loading models to device...
🧹 Memory cleaned
✅ Query encoder loaded. Memory: MPS: Memory tracking not available
✅ Doc encoder loaded. Memory: MPS: Memory tracking not available
🧹 Memory cleaned
🚀 Ready to train! Final memory: MPS: Memory tracking not available


In [ ]:
# --- Memory-Optimized Training Loop with Gradient Accumulation ---
import time

# Training configuration for memory optimization
GRADIENT_ACCUMULATION_STEPS = 4  # Accumulate gradients over 4 steps
MEMORY_CLEANUP_FREQUENCY = 100   # Clean memory every 100 batches

print("🚀 Starting memory-optimized training...")
print(f"   📊 Gradient accumulation steps: {GRADIENT_ACCUMULATION_STEPS}")
print(f"   🧹 Memory cleanup every {MEMORY_CLEANUP_FREQUENCY} batches")
start_time = time.time()

for epoch in range(config['EPOCHS']):
    epoch_start = time.time()
    total_loss = 0
    num_batches = 0
    accumulation_loss = 0
    
    # Set models to training mode
    query_encoder.train()
    doc_encoder.train()
    
    for batch_idx, (query_batch, pos_batch, neg_batch) in enumerate(train_loader):
        try:
            # Move tensors to device
            query_batch = query_batch.to(device, non_blocking=True)
            pos_batch = pos_batch.to(device, non_blocking=True)
            neg_batch = neg_batch.to(device, non_blocking=True)
            
            # Forward pass
            q_vec = query_encoder(query_batch)
            pos_vec = doc_encoder(pos_batch)
            neg_vec = doc_encoder(neg_batch)

            loss = triplet_loss_function((q_vec, pos_vec, neg_vec), F.pairwise_distance, config['MARGIN'])
            
            # Scale loss for gradient accumulation
            loss = loss / GRADIENT_ACCUMULATION_STEPS
            
            # Backward pass
            loss.backward()
            
            accumulation_loss += loss.item()
            
            # Update weights every GRADIENT_ACCUMULATION_STEPS
            if (batch_idx + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
                optimizer.step()
                optimizer.zero_grad()
                
                total_loss += accumulation_loss * GRADIENT_ACCUMULATION_STEPS
                accumulation_loss = 0
            
            num_batches += 1
            
            # Progress indicator and memory management
            if num_batches % 50 == 0:
                current_loss = loss.item() * GRADIENT_ACCUMULATION_STEPS
                print(f"  Batch {num_batches}/{len(train_loader)}, Loss: {current_loss:.4f}, Memory: {get_memory_usage()}")
            
            # Periodic memory cleanup
            if num_batches % MEMORY_CLEANUP_FREQUENCY == 0:
                clean_memory()
                
            # Clear intermediate tensors
            del query_batch, pos_batch, neg_batch, q_vec, pos_vec, neg_vec, loss
            
        except RuntimeError as e:
            if "out of memory" in str(e):
                print(f"❌ Memory error at batch {num_batches}: {str(e)}")
                print("🧹 Attempting memory cleanup and continuing...")
                clean_memory()
                optimizer.zero_grad()  # Clear gradients
                continue
            else:
                raise e
    
    # Handle any remaining accumulated gradients
    if accumulation_loss > 0:
        optimizer.step()
        optimizer.zero_grad()
        total_loss += accumulation_loss * GRADIENT_ACCUMULATION_STEPS
    
    epoch_time = time.time() - epoch_start
    avg_loss = total_loss / max(num_batches, 1)
    print(f"✅ Epoch {epoch+1}/{config['EPOCHS']}, Avg Loss: {avg_loss:.4f}, Time: {epoch_time:.1f}s")
    print(f"   Memory after epoch: {get_memory_usage()}")
    
    # Clean memory after each epoch
    clean_memory()

total_time = time.time() - start_time
print(f"\n🎉 Training completed! Total time: {total_time/60:.1f} minutes")
print(f"Final memory state: {get_memory_usage()}")

🚀 Starting memory-optimized training...
   📊 Gradient accumulation steps: 4
   🧹 Memory cleanup every 100 batches
  Batch 50/404366, Loss: 1.6608, Memory: MPS: Memory tracking not available
  Batch 100/404366, Loss: 0.9258, Memory: MPS: Memory tracking not available
🧹 Memory cleaned
  Batch 150/404366, Loss: 1.0752, Memory: MPS: Memory tracking not available
  Batch 200/404366, Loss: 0.9413, Memory: MPS: Memory tracking not available
🧹 Memory cleaned
  Batch 250/404366, Loss: 1.0299, Memory: MPS: Memory tracking not available
  Batch 300/404366, Loss: 1.3037, Memory: MPS: Memory tracking not available
🧹 Memory cleaned
  Batch 350/404366, Loss: 1.5755, Memory: MPS: Memory tracking not available
  Batch 400/404366, Loss: 0.9334, Memory: MPS: Memory tracking not available
🧹 Memory cleaned
  Batch 450/404366, Loss: 0.8289, Memory: MPS: Memory tracking not available
  Batch 500/404366, Loss: 1.0617, Memory: MPS: Memory tracking not available
🧹 Memory cleaned
  Batch 550/404366, Loss: 0.6967

In [ ]:
# 🧪 TEST: Verify Environment and Memory Management
print("🧪 Testing memory management setup...")
print(f"✅ PYTORCH_MPS_HIGH_WATERMARK_RATIO = {os.environ.get('PYTORCH_MPS_HIGH_WATERMARK_RATIO', 'NOT SET')}")
print(f"✅ Current device: {device}")
print(f"✅ Current memory state: {get_memory_usage()}")

# Test memory cleaning
print("\n🧹 Testing memory cleaning...")
clean_memory()
print("✅ Memory cleaning completed")

# Test a small forward pass to verify everything works
print("\n🔬 Testing small forward pass...")
try:
    with torch.no_grad():
        # Create small test tensors
        test_query = torch.randint(1, 1000, (1, 10)).to(device)
        test_doc = torch.randint(1, 1000, (1, 15)).to(device)
        
        # Forward pass
        q_vec = query_encoder(test_query)
        d_vec = doc_encoder(test_doc)
        
        print(f"✅ Forward pass successful!")
        print(f"   Query vector shape: {q_vec.shape}")
        print(f"   Doc vector shape: {d_vec.shape}")
        
        # Cleanup
        del test_query, test_doc, q_vec, d_vec
        clean_memory()
        
except Exception as e:
    print(f"❌ Forward pass failed: {str(e)}")

print("\n🎯 Setup verification complete! You can now run the training cell.")


In [21]:
# --- Automatic Model Saving ---
import os
import json
from datetime import datetime

# Create artifacts directory
artifacts_dir = "../artifacts"
os.makedirs(artifacts_dir, exist_ok=True)

# Create timestamped run directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_dir = os.path.join(artifacts_dir, f"two_tower_run_{timestamp}")
os.makedirs(run_dir, exist_ok=True)

print(f"💾 Saving artifacts to: {run_dir}")

# Save model state dictionaries
torch.save({
    'query_encoder_state_dict': query_encoder.state_dict(),
    'doc_encoder_state_dict': doc_encoder.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': config['EPOCHS'],
    'final_loss': avg_loss
}, os.path.join(run_dir, 'model_checkpoint.pth'))

# Save model architectures (for easy loading later)
torch.save(query_encoder, os.path.join(run_dir, 'query_encoder_full.pth'))
torch.save(doc_encoder, os.path.join(run_dir, 'doc_encoder_full.pth'))

# Save training configuration
training_config = {
    'model_config': {
        'vocab_size': VOCAB_SIZE,
        'embed_dim': EMBED_DIM,
        'hidden_dim': config['HIDDEN_DIM'],
        'margin': config['MARGIN']
    },
    'training_config': {
        'epochs': config['EPOCHS'],
        'batch_size': config['BATCH_SIZE'],
        'learning_rate': config['LR'],
        'device': str(device)
    },
    'data_config': {
        'train_samples': len(train_data),
        'val_samples': len(val_data),
        'test_samples': len(test_data),
        'total_triplets': len(train_data) + len(val_data) + len(test_data)
    },
    'training_results': {
        'final_avg_loss': avg_loss
    }
}

with open(os.path.join(run_dir, 'training_config.json'), 'w') as f:
    json.dump(training_config, f, indent=2)

# Save tokenizer (copy the word2idx file or save the object)
import shutil
if os.path.exists(config['WORD_TO_IDX_PATH']):
    shutil.copy2(config['WORD_TO_IDX_PATH'], os.path.join(run_dir, 'word_to_idx.pkl'))

print(f"✅ Saved artifacts:")
print(f"  📁 Directory: {run_dir}")
print(f"  🧠 Models: model_checkpoint.pth, *_encoder_full.pth")
print(f"  ⚙️  Config: training_config.json")
print(f"  📝 Tokenizer: word_to_idx.pkl")

In [23]:
# --- MPS-Optimized Inference Function ---
def search(query_text, documents, tokenizer, query_encoder, doc_encoder):
    """
    Search function optimized for MPS (Apple Silicon GPU)
    """
    # Get device from encoder
    device = next(query_encoder.parameters()).device
    
    with torch.no_grad():
        # Encode query and move to device
        query_tensor = pad_sequence([torch.tensor(tokenizer.encode(query_text), dtype=torch.long)], batch_first=True).to(device)
        query_vec = query_encoder(query_tensor)

        # Encode documents and move to device
        doc_tensors = pad_sequence([torch.tensor(tokenizer.encode(doc), dtype=torch.long) for doc in documents], batch_first=True).to(device)
        doc_vecs = doc_encoder(doc_tensors)

        # Calculate similarity scores
        scores = F.cosine_similarity(query_vec, doc_vecs)
        top_indices = torch.argsort(scores, descending=True)
        
        # Convert results back to CPU for return (if needed)
        results = [(documents[i], scores[i].item()) for i in top_indices]
        
        # Clear MPS cache after inference if available
        if device.type == 'mps' and hasattr(torch.mps, 'empty_cache'):
            torch.mps.empty_cache()
            
        return results

Loaded pretrained embeddings: (400000, 200)
Vocabulary size: 400000
Embedding dimension: 200
Using device: cpu


In [ ]:
# --- Comprehensive Testing with Real Data ---
import random
from collections import defaultdict

def evaluate_retrieval(test_data, query_encoder, doc_encoder, tokenizer, k=10):
    """
    Evaluate retrieval performance using real test data
    """
    print("🔍 COMPREHENSIVE RETRIEVAL EVALUATION")
    print("="*50)
    
    # Group test data by query to get all relevant docs per query
    query_to_docs = defaultdict(list)
    for query, pos_doc, neg_doc in test_data[:100]:  # Sample 100 for speed
        query_to_docs[query].extend([pos_doc, neg_doc])
    
    # Test multiple queries
    sample_queries = list(query_to_docs.keys())[:5]  # Test 5 queries
    
    for i, query in enumerate(sample_queries):
        print(f"\n🔎 TEST QUERY {i+1}: {query[:100]}...")
        print("-" * 60)
        
        # Get all documents for this query
        documents = query_to_docs[query]
        
        # Add some random documents from other queries for harder test
        other_docs = []
        for other_query in random.sample(list(query_to_docs.keys()), 3):
            if other_query != query:
                other_docs.extend(query_to_docs[other_query][:2])
        
        all_documents = documents + other_docs
        random.shuffle(all_documents)
        
        print(f"📚 Searching through {len(all_documents)} documents...")
        
        # Run search
        results = search(query, all_documents, tokenizer, query_encoder, doc_encoder)
        
        print(f"\n🏆 TOP {min(3, len(results))} RESULTS:")
        for j, (doc, score) in enumerate(results[:3]):
            relevance = "✅ RELEVANT" if doc in documents else "❌ NOT RELEVANT"
            print(f"{j+1}. Score: {score:.4f} {relevance}")
            print(f"   Doc: {doc[:80]}...")
            print()

# Run comprehensive evaluation
evaluate_retrieval(test_data, query_encoder, doc_encoder, tokenizer)

🚀 Starting training...
  Batch 50/126365, Loss: 1.0012
  Batch 100/126365, Loss: 0.9825


In [ ]:

# --- Automatic Model Saving ---
import os
import json
from datetime import datetime

# Create artifacts directory
artifacts_dir = "../artifacts"
os.makedirs(artifacts_dir, exist_ok=True)

# Create timestamped run directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_dir = os.path.join(artifacts_dir, f"two_tower_run_{timestamp}")
os.makedirs(run_dir, exist_ok=True)

print(f"💾 Saving artifacts to: {run_dir}")

# Save model state dictionaries
torch.save({
    'query_encoder_state_dict': query_encoder.state_dict(),
    'doc_encoder_state_dict': doc_encoder.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': config['EPOCHS'],
    'final_loss': avg_loss
}, os.path.join(run_dir, 'model_checkpoint.pth'))

# Save model architectures (for easy loading later)
torch.save(query_encoder, os.path.join(run_dir, 'query_encoder_full.pth'))
torch.save(doc_encoder, os.path.join(run_dir, 'doc_encoder_full.pth'))

# Save training configuration
training_config = {
    'model_config': {
        'vocab_size': VOCAB_SIZE,
        'embed_dim': EMBED_DIM,
        'hidden_dim': config['HIDDEN_DIM'],
        'margin': config['MARGIN']
    },
    'training_config': {
        'epochs': config['EPOCHS'],
        'batch_size': config['BATCH_SIZE'],
        'learning_rate': config['LR'],
        'device': str(device)
    },
    'data_config': {
        'train_samples': len(train_data),
        'val_samples': len(val_data),
        'test_samples': len(test_data),
        'total_triplets': len(train_data) + len(val_data) + len(test_data)
    },
    'training_results': {
        'final_avg_loss': avg_loss
    }
}

with open(os.path.join(run_dir, 'training_config.json'), 'w') as f:
    json.dump(training_config, f, indent=2)

# Save tokenizer (copy the word2idx file or save the object)
import shutil
if os.path.exists(config['WORD_TO_IDX_PATH']):
    shutil.copy2(config['WORD_TO_IDX_PATH'], os.path.join(run_dir, 'word_to_idx.pkl'))

print(f"✅ Saved artifacts:")
print(f"  📁 Directory: {run_dir}")
print(f"  🧠 Models: model_checkpoint.pth, *_encoder_full.pth")
print(f"  ⚙️  Config: training_config.json")
print(f"  📝 Tokenizer: word_to_idx.pkl")

In [15]:
# --- MPS-Optimized Inference Function ---
def search(query_text, documents, tokenizer, query_encoder, doc_encoder):
    """
    Search function optimized for MPS (Apple Silicon GPU)
    """
    # Get device from encoder
    device = next(query_encoder.parameters()).device
    
    with torch.no_grad():
        # Encode query and move to device
        query_tensor = pad_sequence([torch.tensor(tokenizer.encode(query_text), dtype=torch.long)], batch_first=True).to(device)
        query_vec = query_encoder(query_tensor)

        # Encode documents and move to device
        doc_tensors = pad_sequence([torch.tensor(tokenizer.encode(doc), dtype=torch.long) for doc in documents], batch_first=True).to(device)
        doc_vecs = doc_encoder(doc_tensors)

        # Calculate similarity scores
        scores = F.cosine_similarity(query_vec, doc_vecs)
        top_indices = torch.argsort(scores, descending=True)
        
        # Convert results back to CPU for return (if needed)
        results = [(documents[i], scores[i].item()) for i in top_indices]
        
        # Clear MPS cache after inference if available
        if device.type == 'mps' and hasattr(torch.mps, 'empty_cache'):
            torch.mps.empty_cache()
            
        return results

In [ ]:
# --- Comprehensive Testing with Real Data ---
import random
from collections import defaultdict

def evaluate_retrieval(test_data, query_encoder, doc_encoder, tokenizer, k=10):
    """
    Evaluate retrieval performance using real test data
    """
    print("🔍 COMPREHENSIVE RETRIEVAL EVALUATION")
    print("="*50)
    
    # Group test data by query to get all relevant docs per query
    query_to_docs = defaultdict(list)
    for query, pos_doc, neg_doc in test_data[:100]:  # Sample 100 for speed
        query_to_docs[query].extend([pos_doc, neg_doc])
    
    # Test multiple queries
    sample_queries = list(query_to_docs.keys())[:5]  # Test 5 queries
    
    for i, query in enumerate(sample_queries):
        print(f"\n🔎 TEST QUERY {i+1}: {query[:100]}...")
        print("-" * 60)
        
        # Get all documents for this query
        documents = query_to_docs[query]
        
        # Add some random documents from other queries for harder test
        other_docs = []
        for other_query in random.sample(list(query_to_docs.keys()), 3):
            if other_query != query:
                other_docs.extend(query_to_docs[other_query][:2])
        
        all_documents = documents + other_docs
        random.shuffle(all_documents)
        
        print(f"📚 Searching through {len(all_documents)} documents...")
        
        # Run search
        results = search(query, all_documents, tokenizer, query_encoder, doc_encoder)
        
        print(f"\n🏆 TOP {min(3, len(results))} RESULTS:")
        for j, (doc, score) in enumerate(results[:3]):
            relevance = "✅ RELEVANT" if doc in documents else "❌ NOT RELEVANT"
            print(f"{j+1}. Score: {score:.4f} {relevance}")
            print(f"   Doc: {doc[:80]}...")
            print()

# Run comprehensive evaluation
evaluate_retrieval(test_data, query_encoder, doc_encoder, tokenizer)